In [1]:



# def set_model(triads_species_name, ens_tree):
#     tree_topology_triads = ens_tree.get_sub_tree(triads_species_name.values())
#     null = get_app("model", "TN93", tree=tree_topology_triads, time_het="max", optimise_motif_probs=True)
#     alt = get_app("model", "GN", tree=tree_topology_triads, time_het="max", optimise_motif_probs=True)
#     hyp = get_app("hypothesis", null, alt)
#     return hyp

# def get_model_fitting_result(triads_species_name, triads_aln, ens_tree):
#     # ingroup_species_gene_name = [triads_species_name['ingroup1'], triads_species_name['ingroup2']]
#     # outgroup = triads_species_name['outgroup']
#     model = set_model(triads_species_name, ens_tree)
#     model_fitting_result = model(triads_aln)
#     return model_fitting_result


# def process_path(path, result_lf_dir, triads_info_dir):
#     model_fitting_result_dict = {}
#     file_name = os.path.basename(path.rstrip('/'))
#     print(file_name)
#     triads_alignment_paths = glob.glob(os.path.join(path, "*.json"))
#     triads_info_path = os.path.join(triads_info_dir, file_name, "triads_species_names_dict.json")
#     triads_speccies_infos = json.load((open(triads_info_path, 'r')))
#     result_lf_path = os.path.join(result_lf_dir, f"{file_name}.json")
#     result_lf = load_json_app(result_lf_path)
    
#     for alignment_path in triads_alignment_paths:
#         identifier_number = os.path.basename(alignment_path).rsplit('.', 1)[0]
#         print(identifier_number)
#         triads_aln = deserialise_object(json.load(open(alignment_path, 'r')))
#         triads_species_name = triads_speccies_infos[identifier_number]
#         print(triads_species_name)
#         ens_tree = result_lf.get_ens_tree()
#         model_fitting_result= get_model_fitting_result(triads_species_name, triads_aln, ens_tree)
#         model_fitting_result_dict[identifier_number] = model_fitting_result

#     return model_fitting_result_dict

In [1]:
from cogent3.util.deserialise import deserialise_object
import json
import os
import glob
from cogent3 import get_app, open_data_store
from clock_project.simulation.wts import calculate_non_stationarity
import numpy as np
from cogent3.evolve.models import register_model
from cogent3.evolve.ns_substitution_model import GeneralStationary
from cogent3 import make_tree, get_moltype
from cogent3.app.composable import LOADER, define_app
from cogent3.app.typing import AlignedSeqsType, HypothesisResultType, BootstrapResultType, SerialisableType, IdentifierType

from cogent3.app import evo
RATE_PARAM_UPPER = 50
load_json_app = get_app("load_json")

@register_model("nucleotide")
def GSN(**kwargs):
    """A General Stationary Nucleotide substitution model instance."""
    kwargs["optimise_motif_probs"] = kwargs.get("optimise_motif_probs", True)
    kwargs["name"] = kwargs.get("name", "GSN")
    return GeneralStationary(get_moltype("dna").alphabet, **kwargs)

@register_model("nucleotide")
def GN_max_het(**kwargs):
    """A General Stationary Nucleotide substitution model instance."""
    kwargs["optimise_motif_probs"] = kwargs.get("optimise_motif_probs", True)
    kwargs["time_het"] = kwargs.get("time_het", 'max')
    kwargs["name"] = kwargs.get("name", "GN_max_het")
    return GeneralStationary(get_moltype("dna").alphabet, **kwargs)


def get_param_rules_upper_limit(model_name, upper):
    """rules to set the upper value for rate matrix terms"""
    from cogent3 import get_model

    sm = get_model(model_name)
    return [{"par_name": par_name, "upper": upper} for par_name in sm.get_param_list()]

def test_hypothesis_model_bootstrapper(aln: AlignedSeqsType, tree=None, opt_args=None) -> HypothesisResultType:
    
    outgroup_name = aln.info['triads_species_name']['outgroup']
    tree = make_tree(tip_names=aln.names)
    
    outgroup_edge = [outgroup_name]

    model_kwargs = dict(
    tree=tree,
    opt_args=opt_args,
    # unique_trees=True,
    lf_args=dict(discrete_edges=[outgroup_edge]),
    optimise_motif_probs=True,
    )
    null = evo.model(
            "GSN",
            param_rules=get_param_rules_upper_limit("GSN", RATE_PARAM_UPPER),
            time_het = None,
            **model_kwargs,
        )
    alt = evo.model(
            "GN",
            param_rules=get_param_rules_upper_limit("GN", RATE_PARAM_UPPER),
            time_het = None,
            **model_kwargs,
        )
    
    alt2 = evo.model(
            "GN",
            name="GN-max-het",
            param_rules=get_param_rules_upper_limit("GN", RATE_PARAM_UPPER),
            time_het = 'max',
            **model_kwargs,
        )
    
    
    models = [null, alt, alt2]
    
    return evo.model_collection(*models, sequential=True)(aln)
    

def p_value(result):
    return sum(result.observed.LR <= null_lr for null_lr in result.null_dist) / len(result.null_dist)


aln_dir_new = '/Users/gulugulu/Desktop/honours/data_local/triples_aln_subset_info_added'

# bootstrapper = test_hypothesis_model_bootstrapper()
# path_bootstrapper = load_json_app + bootstrapper

input_data_store = open_data_store(aln_dir_new, suffix= 'json', limit=3)
aln = load_json_app(input_data_store[1])


In [2]:
a = test_hypothesis_model_bootstrapper(aln)

In [3]:
a.get_hypothesis_result('GN', 'GN-max-het')

Statistics
==========================
      LR    df      pvalue
--------------------------
223.3281    22    2.91e-35
--------------------------
===================================================================
hypothesis    key                    lnL    nfp    DLC     unique_Q
-------------------------------------------------------------------
null          'GN'            -4583.6686     17    True    True    
alt           'GN-max-het'    -4472.0045     39    True    True    
-------------------------------------------------------------------

In [4]:
a.get_hypothesis_result('GSN', 'GN')

Statistics
=========================
     LR    df      pvalue
-------------------------
63.8013     2    1.40e-14
-------------------------
============================================================
hypothesis    key             lnL    nfp    DLC     unique_Q
------------------------------------------------------------
null          'GSN'    -4615.5692     15    True    True    
alt           'GN'     -4583.6686     17    True    True    
------------------------------------------------------------

In [6]:
# out_dir= '/Users/gulugulu/Desktop/honours/data_local/triples_aln_subset_info_added'
# out_dstore = open_data_store(out_dir, mode="w", suffix="json")
# write_json_app = get_app("write_json", data_store=out_dstore)

# def add_species_name_info(aln_dir, all_info):
#     aln_paths = glob.glob(os.path.join(aln_dir, '*.json'))
#     for path in aln_paths:
#         aln = deserialise_object(json.load(open(path, 'r')))
#         gene_name_identifier = path.split('/')[-1].split('-')[0].split('.')[-2]
#         gene_name = gene_name_identifier.split('_')[-2]
#         identifier = gene_name_identifier.split('_')[-1]
#         triads_species_name = all_info[gene_name][identifier]
#         aln.info['triads_species_name'] = triads_species_name
#         file_identifier = gene_name_identifier + ".json"
#         write_json_app(aln, identifier=file_identifier)


In [43]:
# result_lr = {}
# result_pvalue = {}
# for path in glob.glob(os.path.join(aln_dir, '*.json')):
#     gene_name_identifier = path.split('/')[-1].split('-')[0].split('.')[-2]
#     gene_name = gene_name_identifier.split('_')[-2]
#     identifier = gene_name_identifier.split('_')[-1]
#     triads_species_name = all_info[gene_name][identifier]
#     triads_aln = deserialise_object(json.load(open(path, 'r')))
#     result = fit_hypothesis_model(triads_aln, triads_species_name)(triads_aln)
#     keys = list(result_lr.keys())
#     if gene_name in keys:
#         result_lr[gene_name].append(result.LR)
#         result_pvalue[gene_name].append(result_pvalue)
#     else:
#         result_lr[gene_name] = [result.LR]
#         result_pvalue[gene_name] = [result_pvalue]



In [21]:
from cogent3.evolve import ns_substitution_model, substitution_model

@define_app
def test_hypothesis_clock_model(aln_path: IdentifierType, tree=None, opt_args=None) -> HypothesisResultType:
    aln = load_json_app(aln_path)
    # result_lf = load_json_app(result_lf_path)
    outgroup_name = aln.info['triads_species_name']['outgroup']
    outgroup_edge = [outgroup_name]

    tree = make_tree(tip_names=aln.names)

    model_kwargs = dict(
    tree=tree,
    opt_args=opt_args,
    # unique_trees=True,
    lf_args=dict(discrete_edges=[outgroup_edge]),
    optimise_motif_probs=True,
    )
    null = evo.model(
            "GN",
            param_rules=get_param_rules_upper_limit("GN", RATE_PARAM_UPPER),
            time_het = None,
            **model_kwargs,
        )
    alt = evo.model(
            "GN",
            name="GN-max-het",
            param_rules=get_param_rules_upper_limit("GN", RATE_PARAM_UPPER),
            time_het = "max",
            **model_kwargs,
        )
    
    hyp = evo.hypothesis(null, alt, sequential=True)
    result = hyp(aln)

    return result

In [22]:
hypothesis_clock_model_test = test_hypothesis_clock_model()

In [24]:
aln_dir = '/Users/gulugulu/Desktop/honours/data_local/triples_aln_subset_info_added'
result_lf_path = '/Users/gulugulu/repos/PuningAnalysis/results/output_data/model_fitting_result_350_threshold/ENSG00000007933.json'
clock_test_p_value = []
clock_test_LR_value = []
aln_paths = glob.glob(os.path.join(aln_dir, '*.json'))

for path in aln_paths:
    result = hypothesis_clock_model_test(path)
    clock_test_p_value.append(result.pvalue)
    clock_test_LR_value.append(result.LR)



In [29]:
path1 = aln_paths[0]
result = hypothesis_clock_model_test(path1)

In [39]:
null = result.null.lf
alt = result.alt.lf

In [42]:
aln = load_json_app(path1)

In [46]:
aln.probs_per_seq()

Ferret,0.1710,0.2867,0.3034,0.2389
Cat,0.1925,0.2575,0.2791,0.2708
Horse,0.1962,0.2541,0.2799,0.2698


In [40]:
alt.get_motif_probs_by_node()

root,0.2910,0.2386,0.2007,0.2698
Ferret,0.2389,0.2867,0.1710,0.3034
Cat,0.2708,0.2575,0.1925,0.2791
Horse,0.2698,0.2541,0.1962,0.2799


In [38]:
null.get_motif_probs_by_node()

root,0.2923,0.2371,0.2024,0.2681
Ferret,0.2539,0.2713,0.1837,0.2911
Cat,0.2699,0.2573,0.1915,0.2814
Horse,0.2555,0.2699,0.1844,0.2901


In [32]:
len([x for x in clock_test_p_value if x < 0.25])


1575

In [31]:
clock_test_p_value

[1.7574675105520326e-12,
 2.3607974372779924e-51,
 0.9981507350306227,
 0.7202674045417199,
 0.2086285981511348,
 0.6677678733124695,
 0.03045909858707063,
 0.0011790153287020721,
 0.0005615828196322956,
 0.2425330523624268,
 0.17056657516644264,
 5.100991691852017e-25,
 0.08854665482733588,
 0.026154349276607168,
 4.901620938532484e-54,
 0.6818593092910181,
 9.044514057496413e-08,
 0.009057678284850202,
 0.007323942364311128,
 2.321175106138927e-05,
 0.00018064658416947976,
 0.15319784433621103,
 0.2377208600192297,
 0.10838783936360408,
 0.012491822094243817,
 0.14985797481933652,
 0.7298150200636193,
 0.2330443225292283,
 0.06630076797464624,
 4.621894168537187e-08,
 0.013331347123241076,
 0.868651141519631,
 0.3336437184730595,
 0.1486585744944721,
 0.0008483019518945725,
 0.7676699331685874,
 0.2263263982974285,
 5.630881419109885e-21,
 2.3625208654189354e-13,
 0.0001354941980776324,
 0.003978687507419906,
 1.1874536471768835e-06,
 0.10197780118451832,
 0.0002688803188396135,
 0.1

In [26]:
clock_test_LR_value

[103.21954868287139,
 303.5180860710243,
 7.558143401984353,
 17.757465764293556,
 27.06870477459961,
 18.634638972123867,
 35.987625736771406,
 47.72614680629613,
 50.139195881096384,
 26.215776784513764,
 28.1560647749111,
 170.86122012409396,
 31.38268334438635,
 36.600770053522865,
 316.71809459138785,
 18.402763571807554,
 75.4190358178239,
 40.6540796557606,
 41.42898091652296,
 59.888740347529165,
 53.710545897356496,
 28.714368059952903,
 26.331516483800897,
 30.42974976438336,
 39.460212227575994,
 28.8272962846504,
 17.593227335758456,
 26.445557798063874,
 32.69461254590715,
 77.2177942382109,
 39.21527428613081,
 14.848643592905546,
 24.26206882901397,
 28.868330950079326,
 48.80567308677928,
 16.921676313006174,
 26.612210277151917,
 149.6253386061412,
 108.14460425676589,
 54.59654832196793,
 43.599376591570945,
 68.37808410941761,
 30.720517686655967,
 52.47291339394087,
 29.841404031582897,
 16.450973740225436,
 36.461711752018346,
 26.46612800692901,
 12.56426946897954

In [ ]:
from clock_project.maths.evolutionary_rate import calculate_stationary_distribution

def get_STI(triads_info):
    matrix_ingroup_dict = {}
    nuc_freq_ingroup_dict = {}
    STI_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        matrix_dict = triads_info_value['matrices']
        nuc_freqs_dict = triads_info_value['nuc_freqs_dict']
        nuc_freqs_ingroup_pair = {triads_names['ingroup1']: nuc_freqs_dict['ingroup1'], triads_names['ingroup2']: nuc_freqs_dict['ingroup2']}
        nuc_freq_ingroup_dict[identifier] = nuc_freqs_ingroup_pair
        matrix_ingroup_pair = {triads_names['ingroup1']: matrix_dict[triads_names['ingroup1']], triads_names['ingroup2']: matrix_dict[triads_names['ingroup2']]}
        matrix_ingroup_dict[identifier] = matrix_ingroup_pair
        stationary_distirbution_dict = {triads_names['ingroup1']: calculate_stationary_distribution(matrix_dict[triads_names['ingroup1']]), triads_names['ingroup2']: calculate_stationary_distribution(matrix_dict[triads_names['ingroup2']])}
        







In [ ]:
from clock_project.maths.evolutionary_rate import calculate_stationary_distribution

def get_STI(triads_info):
    matrix_ingroup_dict = {}
    nuc_freq_ingroup_dict = {}
    STI_dict = {}
    for identifier, info in triads_info.items():
        triads_info_value = info['triads_info_small_tree']
        triads_names = info['triads_species_names']
        matrix_dict = triads_info_value['matrices']
        nuc_freqs_dict = triads_info_value['nuc_freqs_dict']
        nuc_freqs_ingroup_pair = {
            triads_names['ingroup1']: nuc_freqs_dict['ingroup1'],
            triads_names['ingroup2']: nuc_freqs_dict['ingroup2']
        }
        nuc_freq_ingroup_dict[identifier] = nuc_freqs_ingroup_pair
        matrix_ingroup_pair = {
            triads_names['ingroup1']: matrix_dict[triads_names['ingroup1']],
            triads_names['ingroup2']: matrix_dict[triads_names['ingroup2']]
        }
        matrix_ingroup_dict[identifier] = matrix_ingroup_pair
        stationary_distribution_dict = {
            triads_names['ingroup1']: calculate_stationary_distribution(matrix_dict[triads_names['ingroup1']]),
            triads_names['ingroup2']: calculate_stationary_distribution(matrix_dict[triads_names['ingroup2']])
        }
        
        # Calculate STI values for each ingroup species
        for species, freqs in nuc_freqs_ingroup_pair.items():
            π = stationary_distribution_dict[species]
            p = freqs
            ΔC = p[2] - π[2]  # C
            ΔG = p[3] - π[3]  # G
            ΔA = p[1] - π[1]  # A
            ΔT = p[0] - π[0]  # T
            
            STI1 = ΔC + ΔG
            STI2 = ΔA - ΔT
            STI3 = ΔC - ΔG
            
            if identifier not in STI_dict:
                STI_dict[identifier] = {}
            STI_dict[identifier][species] = (STI1, STI2, STI3)

    return STI_dict


In [43]:
path_test = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_alignment_350_threshold/ENSG00000152779'
result_lf_dir_test = '/Users/gulugulu/repos/PuningAnalysis/results/output_data/model_fitting_result_350_threshold'
triads_info_dir_test = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_350_threshold'

In [44]:
model_fitting_result_dict = process_path(path_test, result_lf_dir_test, triads_info_dir_test)

ENSG00000152779
396
{'ingroup1': 'Ferret', 'ingroup2': 'Narwhal', 'outgroup': 'Megabat'}
115
{'ingroup1': 'Sooty_mangabey', 'ingroup2': 'Panamanian_white_faced_capuchin', 'outgroup': 'Tarsier'}
457
{'ingroup1': 'Red_fox', 'ingroup2': 'Cat', 'outgroup': 'Donkey'}
20
{'ingroup1': 'Macaque', 'ingroup2': 'Sumatran_orangutan', 'outgroup': 'Panamanian_white_faced_capuchin'}
98
{'ingroup1': 'Vervet_AGM', 'ingroup2': 'Gorilla', 'outgroup': "Ma's_night_monkey"}
61
{'ingroup1': 'Vervet_AGM', 'ingroup2': 'Arctic_ground_squirrel', 'outgroup': 'Tree_Shrew'}
302
{'ingroup1': 'Cow', 'ingroup2': 'Dog', 'outgroup': 'Megabat'}
181
{'ingroup1': 'Sooty_mangabey', 'ingroup2': 'Long_tailed_chinchilla', 'outgroup': 'Tree_Shrew'}
494
{'ingroup1': 'American_black_bear', 'ingroup2': 'Megabat', 'outgroup': 'Guinea_Pig'}
230
{'ingroup1': 'Squirrel', 'ingroup2': 'Tree_Shrew', 'outgroup': 'Greater_horseshoe_bat'}
16
{'ingroup1': 'Alpaca', 'ingroup2': 'Ferret', 'outgroup': 'Greater_horseshoe_bat'}
267
{'ingroup1': '

In [45]:
model_fitting_result_dict

{'396': Statistics
      LR    df    pvalue
 -----------------------
 59.0506    36    0.0091
 -----------------------
 hypothesis    key              lnL    nfp    DLC     unique_Q
 -------------------------------------------------------------
 null          'TN93'    -1251.4650     15    True    True    
 alt           'GN'      -1221.9397     51    True    True    
 -------------------------------------------------------------,
 '115': Statistics
      LR    df    pvalue
 -----------------------
 35.9832    36    0.4694
 -----------------------
 hypothesis    key              lnL    nfp    DLC     unique_Q
 -------------------------------------------------------------
 null          'TN93'    -1025.9271     15    True    True    
 alt           'GN'      -1007.9355     51    True    True    
 -------------------------------------------------------------,
 '457': Statistics
      LR    df    pvalue
 -----------------------
 51.7731    36    0.0430
 -----------------------
 hypothesis

In [11]:
model_fitting_dir = '/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_info/ENSG00000160050'
model_fitting_result_dir = os.path.join(model_fitting_dir, 'model_fitting_result')
model_fitting_results_paths = glob.glob(os.path.join(model_fitting_result_dir, '*.json'))
model_fitting_results_path ='/Users/gulugulu/Desktop/honours/data_local/whole_genome_mammal87/triads_model_fitting_info/ENSG00000160050/model_fitting_result/100.json'
model_fitting_result = load_json_app(model_fitting_results_path)

In [49]:
p_value_dict = {}
for identifier, result in model_fitting_result_dict.items():
    p_value = result.pvalue
    p_value_dict[identifier] = p_value
    

In [50]:
len(p_value_dict)

100

In [51]:
import plotly.express as px
px.histogram(p_value_dict.values())

In [41]:
len(p_value_dict)

100

In [43]:
valid_triads_identifier = []
load_json_app = get_app("load_json")

for identifier, model_fitting_results in model_fitting_result_alt_dict.items():
    vio = []
    exclude_params = ("length", "mprobs")
    list_of_params = model_fitting_results.get_param_rules()
    for param in list_of_params:
        if param["par_name"] not in exclude_params:
            if (abs(param["init"] - param["lower"]) <= 1e-12) or (
                abs(param["init"] - param["upper"]) <= 1e-12):
                vio.append(param)
    if vio == []:
        valid_triads_identifier.append(identifier)


In [41]:
valid_triads_identifier

['134',
 '40',
 '99',
 '99',
 '99',
 '99',
 '76',
 '102',
 '107',
 '106',
 '13',
 '13',
 '188',
 '183',
 '63',
 '96',
 '23']

In [4]:
model_fitting_result_nul_dict

NameError: name 'model_fitting_result_nul_dict' is not defined

In [15]:
from cogent3 import available_models
available_models(model_types=None)

nucleotide,BH,Barry and Hartigan Discrete Time substitution model Barry and Hartigan 1987. Biometrics 43: 261–76.
nucleotide,DT,"Discrete Time substitution model (non-stationary, non-reversible). motif_length=2 makes this a dinucleotide model, motif_length=3 a trinucleotide model."
nucleotide,GN,"General Markov Nucleotide (non-stationary, non-reversible). Kaehler, Yap, Zhang, Huttley, 2015, Sys Biol 64 (2): 281–93"
nucleotide,ssGN,"strand-symmetric general Markov nucleotide (non-stationary, non-reversible). Kaehler, 2017, Journal of Theoretical Biology 420: 144–51"
nucleotide,K80,Kimura 1980
nucleotide,JC69,Jukes and Cantor's 1969 model
nucleotide,GTR,General Time Reversible nucleotide substitution model.
nucleotide,TN93,Tamura and Nei 1993 model
nucleotide,HKY85,"Hasegawa, Kishino and Yano 1985 model"
nucleotide,F81,Felsenstein's 1981 model
codon,CNFGTR,"Conditional nucleotide frequency codon substitution model, GTR variant (with params analagous to the nucleotide GTR model). Yap, Lindsay, Easteal and Huttley, 2010, Mol Biol Evol 27: 726-734"
